# Getting Data Ready For Model

In [12]:
%run src/imports.py

In [15]:
%run src/getDF.py

/Users/randallbloomquist/Desktop/Desktop/Galvanize/CAPSTONES/Sell Your Home in 30 days/data/redfin_2021-03-14-18-54-03.csv
/Users/randallbloomquist/Desktop/Desktop/Galvanize/CAPSTONES/Sell Your Home in 30 days/data/redfin_2021-03-04-11-49-33.csv
/Users/randallbloomquist/Desktop/Desktop/Galvanize/CAPSTONES/Sell Your Home in 30 days/data/redfin_2021-03-14-18-14-23.csv
/Users/randallbloomquist/Desktop/Desktop/Galvanize/CAPSTONES/Sell Your Home in 30 days/data/redfin_2021-03-14-18-47-54.csv
/Users/randallbloomquist/Desktop/Desktop/Galvanize/CAPSTONES/Sell Your Home in 30 days/data/redfin_2021-03-04-11-50-11.csv
/Users/randallbloomquist/Desktop/Desktop/Galvanize/CAPSTONES/Sell Your Home in 30 days/data/redfin_2021-03-14-18-27-39.csv
/Users/randallbloomquist/Desktop/Desktop/Galvanize/CAPSTONES/Sell Your Home in 30 days/data/redfin_2021-03-14-18-20-18.csv
/Users/randallbloomquist/Desktop/Desktop/Galvanize/CAPSTONES/Sell Your Home in 30 days/data/redfin_2021-03-14-18-26-51.csv
/Users/randallbl

In [21]:
df = pd.read_pickle('data/full.pkl')
df.drop_duplicates(inplace=True)

In [22]:
df['ROOMS'] = df['BEDS'] + df['BATHS']
df = df.drop(['$/SQUARE FEET','BEDS','BATHS','SALE TYPE','PROPERTY TYPE','ADDRESS','CITY','STATE OR PROVINCE','MLS#','STATUS','DAYS ON MARKET'],axis=1)

In [23]:
df

,SOLD DATE,ZIP OR POSTAL CODE,PRICE,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,HOA/MONTH,LATITUDE,LONGITUDE,ROOMS
0,October-15-2020,78255,955000,Reserve At Sonoma Verde,4895.0,33976.0,2019.0,55.0,29.612013,-98.654187,8.5
1,June-25-2019,78255,670000,Reserve At Sonoma Verde,4499.0,20908.0,2008.0,50.0,29.612978,-98.654429,10.5
2,May-24-2019,78256,465000,Stonewall Estates,3641.0,7840.0,2010.0,82.0,29.646189,-98.638447,12.0
3,September-25-2020,78257,345000,The Dominion,1989.0,5662.0,2005.0,230.0,29.647801,-98.614186,4.0
4,June-11-2020,78256,337500,Stonewall Ranch,2166.0,9583.0,2004.0,54.0,29.650304,-98.629082,6.0
...,...,...,...,...,...,...,...,...,...,...,...
223,March-16-2020,78258,275000,Stone Mountain,2251.0,5662.0,1986.0,110.0,29.646371,-98.497260,6.5
224,May-2-2019,78258,335000,Arrowhead,2463.0,7840.0,1994.0,16.0,29.644913,-98.494773,8.0
225,August-28-2020,78258,335000,Arrowhead,3282.0,12196.0,1988.0,23.0,29.644785,-98.493958,7.5
226,February-4-2021,78258,350000,Promontory Pointe,2336.0,7727.0,1999.0,38.0,29.643171,-98.503006,5.5


In [24]:
counts = df.groupby('LOCATION').count()
neighborhoods = counts[counts['SOLD DATE'] > 50].index
neighborhoods_cleaned = df[df['LOCATION'].isin(neighborhoods)]
neighborhoods_cleaned.to_pickle('data/neighborhoods_cleaned.pkl')

In [40]:
neighborhoods_cleaned = pd.read_pickle('data/neighborhoods_cleaned.pkl')
neighborhoods_cleaned.reset_index(inplace=True)
neighborhoods_cleaned.drop('index',axis=1,inplace=True)
#neighborhoods_cleaned = neighborhoods_cleaned.merge(pd.get_dummies(neighborhoods_cleaned['LOCATION'],drop_first=True), how='outer',left_index=True,right_index=True)
neighborhoods_cleaned.drop(['LOCATION'],axis=1,inplace=True)
neighborhoods_cleaned['SOLD DATE'] = pd.to_datetime(neighborhoods_cleaned['SOLD DATE'])
neighborhoods_cleaned['MONTH'] = pd.DatetimeIndex(neighborhoods_cleaned['SOLD DATE']).month
neighborhoods_cleaned['YEAR'] = pd.DatetimeIndex(neighborhoods_cleaned['SOLD DATE']).year
neighborhoods_cleaned.drop(['SOLD DATE'],axis=1,inplace=True)
neighborhoods_cleaned['HOA/MONTH'].fillna(value=0,inplace=True)
neighborhoods_cleaned.dropna(inplace=True)

In [41]:
neighborhoods_cleaned.to_pickle('data/modeldata.pkl')

In [42]:
neighborhoods_cleaned

,ZIP OR POSTAL CODE,PRICE,SQUARE FEET,LOT SIZE,YEAR BUILT,HOA/MONTH,LATITUDE,LONGITUDE,ROOMS,MONTH,YEAR
0,78257,345000,1989.0,5662.0,2005.0,230.0,29.647801,-98.614186,4.0,9,2020
1,78256,337500,2166.0,9583.0,2004.0,54.0,29.650304,-98.629082,6.0,6,2020
2,78256,375500,2969.0,6605.0,2004.0,54.0,29.651842,-98.630650,6.5,3,2021
3,78255,462000,2716.0,9147.0,2014.0,95.0,29.617650,-98.643430,7.0,10,2018
4,78256,624000,3012.0,75794.0,2000.0,25.0,29.620646,-98.622295,5.5,2,2021
...,...,...,...,...,...,...,...,...,...,...,...
20007,78222,153000,1577.0,6969.0,1983.0,0.0,29.383410,-98.376520,5.0,9,2019
20008,78222,189900,1703.0,4791.0,2018.0,25.0,29.386700,-98.379900,6.0,7,2018
20009,78222,128000,1102.0,5662.0,1985.0,0.0,29.383092,-98.376562,5.0,3,2019
20010,78222,196499,2324.0,5662.0,2014.0,6.0,29.379346,-98.370269,7.0,3,2019
